In [1]:
"""
Simulated Annealing for a bounded-integer knapsack (assignment-ready)

Problem
-------
Maximize:  12 x1 + 16 x2 + 22 x3 + 8 x4
Subject to: 4 x1 + 5 x2 + 7 x3 + 3 x4 <= 140
            0 <= xi <= 10, integer

Design (tailored, not boilerplate)
----------------------------------
• State: integer vector x = (x1..x4), each in [0, 10]
• Objective: value(x) = sum(v_i * x_i); hard capacity feasibility
• Neighbor: pick one coordinate i at random, add ±1, then repair
• Repair: clamp to [0, ub_i], then if overweight, decrement items with
          the WORST value/weight ratio first (least painful to drop)
• Start: greedy-by-ratio feasible fill (with a bit of randomness)
• Acceptance: Metropolis (accept worse with prob exp((f(y)-f(x))/T))
• Cooling: geometric T <- alpha * T
• Log: prints per-temperature summary (T, current, best, acceptance)

Parameter rationale (brief)
---------------------------
T0=6.0 gives enough early uphill acceptance; alpha=0.985 cools slowly
for exploration in a small state space; iters_per_T=300 mixes well even
with a simple ±1 neighborhood; Tmin=1e-4 is low enough to "freeze".
"""

import math
import random
from typing import List, Tuple

# ------------ Problem data ------------
values  = [12, 16, 22,  8]   # v_i
weights = [ 4,  5,  7,  3]   # w_i
cap     = 140
ub      = [10, 10, 10, 10]   # 0 <= x_i <= 10 (integers)
n       = len(values)

# ------------ Objective & helpers ------------
def knap_weight(x: List[int]) -> int:
    return sum(w * xi for w, xi in zip(weights, x))

def knap_value(x: List[int]) -> int:
    return sum(v * xi for v, xi in zip(values, x))

def ratio(i: int) -> float:
    """Value-to-weight ratio for item i (domain-aware heuristic)."""
    return values[i] / weights[i]

# ------------ Construction & repair ------------
def greedy_random_start() -> List[int]:
    """
    Ratio-guided feasible start with slight randomness:
    iterate items by v/w (high->low), take as many as fit but
    randomly back off by up to 2 units to diversify starts.
    """
    order = sorted(range(n), key=ratio, reverse=True)
    x = [0] * n
    rem = cap
    for i in order:
        max_take = min(ub[i], rem // weights[i])
        if max_take > 0:
            lo = max(0, max_take - 2)
            take = random.randint(lo, max_take)  # diversify
            x[i] = take
            rem -= take * weights[i]
    return x

def repair(x: List[int]) -> List[int]:
    """
    Enforce bounds and capacity.
    If overweight, decrement items with the LOWEST v/w first
    (least expensive to drop per unit of weight).
    """
    # Clamp to bounds
    x = [max(0, min(ub[i], int(x[i]))) for i in range(n)]

    # Trim until feasible if overweight
    if knap_weight(x) > cap:
        # Items sorted by WORST ratio first
        bad_first = sorted(range(n), key=ratio)  # ascending v/w
        while knap_weight(x) > cap:
            for i in bad_first:
                if x[i] > 0:
                    x[i] -= 1
                    break
    return x

# ------------ Neighborhood ------------
def neighbor(x: List[int]) -> List[int]:
    """
    ±1 change on a random coordinate, then repair to ensure feasibility.
    """
    y = x[:]
    i = random.randrange(n)
    y[i] += random.choice([-1, 1])
    return repair(y)

# ------------ Simulated Annealing core ------------
def anneal(
    T0: float = 6.0,
    Tmin: float = 1e-4,
    alpha: float = 0.985,
    iters_per_T: int = 300,
    max_no_improve: int = 5000,
    seed: int = 2025,
    log: bool = True,
) -> Tuple[List[int], int]:
    random.seed(seed)

    # Construct a strong feasible start
    x = repair(greedy_random_start())
    fx = knap_value(x)

    best_x, best_f = x[:], fx
    T = T0
    no_imp = 0

    # Temperature loop
    while T > Tmin and no_imp < max_no_improve:
        accepted = 0
        for _ in range(iters_per_T):
            y = neighbor(x)
            fy = knap_value(y)
            delta = fy - fx

            # Metropolis acceptance
            if delta >= 0 or random.random() < math.exp(delta / T):
                x, fx = y, fy
                accepted += 1
                if fx > best_f:
                    best_x, best_f = x[:], fx
                    no_imp = 0
                else:
                    no_imp += 1
            else:
                no_imp += 1

        # Per-temperature log (concise)
        if log:
            acc_rate = accepted / iters_per_T
            print(f"T={T:7.4f} | current={fx:4d} | best={best_f:4d} | acc={acc_rate:5.2%}")

        T *= alpha

    return best_x, best_f

# ------------ Run (submission-ready) ------------
if __name__ == "__main__":
    x_best, f_best = anneal(
        T0=6.0, Tmin=1e-4, alpha=0.985,
        iters_per_T=300, max_no_improve=5000,
        seed=2025, log=True
    )

    w_best = knap_weight(x_best)
    print("\nFinal SA solution")
    print(f"  x      = {x_best}")
    print(f"  value  = {f_best}")
    print(f"  weight = {w_best}  (capacity = {cap})")

    # Feasibility assertion for the assignment
    assert min(x_best) >= 0 and max(x_best) <= 10 and w_best <= cap, "Infeasible solution!"

T= 6.0000 | current= 408 | best= 440 | acc=62.67%
T= 5.9100 | current= 436 | best= 440 | acc=65.67%
T= 5.8213 | current= 440 | best= 440 | acc=65.67%
T= 5.7340 | current= 440 | best= 440 | acc=63.67%
T= 5.6480 | current= 440 | best= 440 | acc=63.33%
T= 5.5633 | current= 440 | best= 440 | acc=64.67%
T= 5.4798 | current= 428 | best= 440 | acc=60.67%
T= 5.3977 | current= 440 | best= 440 | acc=62.33%
T= 5.3167 | current= 436 | best= 440 | acc=66.00%
T= 5.2369 | current= 428 | best= 440 | acc=56.00%
T= 5.1584 | current= 440 | best= 440 | acc=63.67%
T= 5.0810 | current= 440 | best= 440 | acc=63.00%
T= 5.0048 | current= 440 | best= 440 | acc=67.00%
T= 4.9297 | current= 440 | best= 440 | acc=62.33%
T= 4.8558 | current= 440 | best= 440 | acc=61.00%
T= 4.7829 | current= 440 | best= 440 | acc=63.00%
T= 4.7112 | current= 440 | best= 440 | acc=63.33%

Final SA solution
  x      = [5, 10, 10, 0]
  value  = 440
  weight = 140  (capacity = 140)
